# Riddles solved with Julia

### Overview

#### 1. [Knights on an infinite chessboard](#p1)
#### 2. [Collecting cigarette cards](#p2)
#### 3. [The weighted dice](#p3)

<br>



In [1]:
using Random
using Statistics
using Memoize
using Base.Iterators

repeat(f::Function, n::Integer) = [f() for _ in 1:n]
nsims = 10^5;
simprob(f) = sum(repeat(f, nsims)) / nsims

Random.seed!(0);

<br>

### <a name="p1"> Knight on an infinite chessboard

**Problem**  
Suppose that a knight makes a “random walk” on an infinite chessboard. Specifically, every turn the knight follows standard chess rules and moves to one of its eight accessible squares, each with probability 1/8.

What is the probability that after the twentieth move the knight is back on its starting square?

**Simulation solution**  
The knight moves two tiles in any direction, and one tile in a direction perpendicular to that. Approximate solution using simulations.

In [2]:
move() = shuffle(1:2) .* rand([-1, 1], 2)
is_back_to_start() = sum(repeat(move, 20)) == [0,0]
simprob(is_back_to_start)

0.00621

**Exact solution**

In [3]:
@memoize function knight_returns(moves_left, i, j)
    moves_left == 0 && return i == j == 0
    n_return = 0
    for(x,y) in product([(1,2),(2,1)], product([-1,1], [-1, 1]))
        n_return += knight_returns(moves_left - 1, i + x[1]*y[1], j + x[2]*y[2])
    end
    n_return
end

knight_returns(20,0,0) / 8^20

0.006208754649023429

<br>

### <a name="p2"> Collecting cigarette cards

**Problem**  
In the video game “Red Dead Redemption 2,” there is a side quest where the main character is supposed to collect 12 sets of cigarette cards, each consisting of 12 unique cards.

Some cards can be found lying around in the open world, but the easiest way to collect the cards is to buy cigarettes at the store and collect the single random card included in each pack. Suppose Arthur is too lazy to look for cards in the open world and tries to complete the set only by buying packs at the store.

At $5 a pack, how much money do we expect Arthur to spend to complete all 12 sets?

**Simulation solution**

In [4]:
function get_price()
    cards, dollars = Set(), 0
    while length(cards) < 144
        push!(cards, rand(1:144))
        dollars += 5
    end
    dollars
end

mean(repeat(get_price, nsims))

3995.1533

**Exact solution**  
This is the [Coupon collector's problem](https://en.wikipedia.org/wiki/Coupon_collector%27s_problem).  
If you are only lacking one card out of 144, it will take an average of 144 packs to get it; if you are lacking two cards, it will take an average of 144 / 2 packs, and so on. So we sum all these averages.

In [5]:
sum(144/n for n in 1:144) * 5

3996.357960919262

<br>

### <a name="p3"> The weighted dice

**Problem**  
You have a friend who you suspect of usign weighted dice. You borrow one of them and roll it 100 times, in which you get 24 sixes. What is the probability that you would've gotten this number of sixes or higher with a normal dice?

**Simuation solution**

In [6]:
twentyfour_sixes() = count(rand(1:6, 100) .== 6) >= 24

simprob(twentyfour_sixes)

0.03818

**Exact solution**

In [7]:
sum(k -> binomial(BigInt(100), BigInt(k)) * (1/6)^k * (5/6)^(100-k), 24:100)

0.03786436865340891793158842474499197681678593666234919466399082464123118584013411